In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, IntegerType, StringType , FloatType
import sys
sys.path.append(r'/home/gabriel/Documents/proj/utils/')
from utils import createSparkSesion
from pyspark.sql import (functions as F,
                                        Window)
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
spark = createSparkSesion()

25/05/21 16:20:53 WARN Utils: Your hostname, GabrielUnbunto resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/05/21 16:20:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/05/21 16:20:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<h1> Handlling Duplicates</h1>
Write a code to find and remove duplicate rows from a df imployees

In [3]:
pip install faker


Note: you may need to restart the kernel to use updated packages.


In [4]:
from faker import Faker
import random

In [5]:
fake = Faker('pt_br')
rand = random

In [6]:
#mocking some data
data   =  []
schema =  StructType([StructField("name", StringType(), False),
                      StructField("cpf", StringType(), False),
                      StructField("id", IntegerType(), False)])
for i in range(100):
   name = fake.name()
   cpf  = fake.cpf()
   id   = i
   if rand.random() >= 0.1 :
      data.append((name,cpf,id))
   else:
      data.append((name,cpf,id))
      data.append((name,cpf,id))


In [7]:
employees_df = spark.createDataFrame(data,schema = schema)

In [8]:
#solving easy mode kkk
employees_df_solution_1 = employees_df.distinct()

print(f'deduped : {employees_df_solution_1.count()}      ;      original:{employees_df.count()}')

deduped : 100      ;      original:113


In [9]:
#alternative mode , could be usefull in other cenarioes , such as , if  nao all the column having the exact same value

dedup_window = Window.partitionBy("name","cpf").orderBy("name")

employees_df_solution_2  = (employees_df
                              .withColumn("rn", F.row_number().over(dedup_window))
                              .filter(F.col("rn") == 1))

print(f'deduped : {employees_df_solution_2.count()}      ;      original:{employees_df.count()}')

deduped : 100      ;      original:113


<h1>NUll value handling</h1>
how would you replace a NULL value in the salary column of the employees table with the average salary

In [10]:
#mocking some data
data   =  []
schema =  StructType([StructField("name", StringType(), False),
                      StructField("cpf", StringType(), False),
                      StructField("id", IntegerType(), False),
                      StructField("salary", FloatType(), True)])
for i in range(100):
   name = fake.name()
   cpf  = fake.cpf()
   id   = i
   salary = rand.uniform(1000 , 10000)
   if rand.random() >= 0.1 :
      data.append((name,cpf,id,salary))
   else :
      data.append((name,cpf,id,None))

In [11]:
null_value_handling_df = spark.createDataFrame(data,schema=schema)

In [ ]:
#solving

avg_salary  = null_value_handling_df.select(F.avg("salary")).collect()[0][0]

null_value_handling_df_solution_1 = null_value_handling_df.withColumn("new_salary",F.when(F.col("salary").isNull(),avg_salary).otherwise(F.col("salary")))

null_value_handling_df_solution_1.filter(F.col("salary").isNull()).takePandas(20)

,name,cpf,id,salary,new_salary
0,Isadora Ribeiro,265.809.317-95,2,None,5953.355948
1,Sr. João Vieira,958.026.471-67,9,None,5953.355948
2,Isaac Nunes,482.673.501-44,10,None,5953.355948
3,Lucas Gabriel Brito,634.972.180-22,22,None,5953.355948
4,Pietro da Rosa,827.364.509-65,28,None,5953.355948
5,Gael Sá,816.490.752-94,31,None,5953.355948
6,Maria Helena Barros,436.582.071-08,37,None,5953.355948
7,Ryan Cirino,796.125.843-46,49,None,5953.355948
8,Beatriz Jesus,781.603.495-48,65,None,5953.355948
9,Diego Marques,407.315.962-34,66,None,5953.355948


In [14]:
null_value_handling_df.count_nulls()

,name,cpf,id,salary
0,0,0,0,15


In [16]:
#solving 2

avg_salary  = null_value_handling_df.select(F.avg("salary")).collect()[0][0]

fill_na_json = {"salary" : avg_salary}

null_value_handling_df_solution_2 = null_value_handling_df.fillna(fill_na_json)

null_value_handling_df_solution_2.count_nulls()

,name,cpf,id,salary
0,0,0,0,0
